<a href="https://colab.research.google.com/github/ffg-kom/mapping_evolution/blob/master/getting_data_ready.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

The following code was copied from https://colab.research.google.com/github/nestauk/im_tutorials/blob/master/notebooks/01_intro_to_pandas_tutorial.ipynb.

In [1]:
%load_ext autoreload
%autoreload 2
# install im_tutorial package
!pip install git+https://github.com/nestauk/im_tutorials.git

  Cloning https://github.com/nestauk/im_tutorials.git to /tmp/pip-req-build-zc0h1xai
  Running command git clone -q https://github.com/nestauk/im_tutorials.git /tmp/pip-req-build-zc0h1xai
  Created wheel for im-tutorials: filename=im_tutorials-0.1.1-cp36-none-any.whl size=15103 sha256=92e91cad0eb6ee4addfa2387327c4dda423190d004e050088169dd06e70e0ce5
  Stored in directory: /tmp/pip-ephem-wheel-cache-lp313cnr/wheels/47/a3/cb/bdc5f9ba49bcfd2c6864b166a1566eb2f104113bf0c3500330
Successfully built im-tutorials


In [0]:
# numpy for mathematical functions
import numpy as np
# pandas for handling tabular data
import pandas as pd
# explained later
from im_tutorials.data.cordis import h2020_projects

# for plotting
import matplotlib.pyplot as plt

# data
from im_tutorials.data.cordis import cordis_table

# Loading data

In [76]:
# load tables
cordis_o_df = cordis_table('organisations')
cordis_p_df = cordis_table('projects')
cordis_p_o_df = cordis_table('project_organisations')

# merge them
cordis_project_orgs_df = cordis_p_o_df.merge(
  cordis_o_df, left_on='organization_id', right_on='id', how='left'
)
cordis_project_orgs_df = cordis_project_orgs_df.merge(
  cordis_p_df, left_on='project_rcn', right_on='rcn', how='left'
)

cordis_project_orgs_df.rename( columns={'contribution': 'ec_contribution_organization'
                                        , 'ec_contribution': 'ec_contribution_project'
                                        , 'total_cost': 'total_cost_project'
                                       }, inplace=True )

cordis_project_orgs_df.head(2)

,project_rcn,organization_id,activity_type,address,ec_contribution_organization,type,website_x,id,name,country_code,country_name,rcn,acronym,end_date_code,ec_contribution_project,framework,funding_scheme,funded_under,objective,project_description,start_date_code,status,title,total_cost_project,website_y
0,85231,999635926,Other,"{'city': 'TORINO', 'street': 'CORSO DUCA DEGLI...",306448,participant,www.corep.it,999635926,CONSORZIO PER LA RICERCA E L EDUCAZIONE PERMAN...,IT,Italy,85231,EuroTraining,2010-10-31,1398009,FP7,CSA - Coordination and support action,"[{'rcn': '853', 'title': 'Specific Programme ""...",The objective of the EuroTraining proposal is ...,\nNext-Generation Nanoelectronics Components a...,2007-11-01,CLOSED,Provision of a European training infrastructure,1398009,http://www.eurotraining.net/
1,85231,999665802,Private for-profit entities (excluding Higher ...,"{'city': 'HOLTE', 'street': 'FREDERIKSLUNDSVEJ...",396542,coordinator,,999665802,TECHNOCONSULT APS,DK,Denmark,85231,EuroTraining,2010-10-31,1398009,FP7,CSA - Coordination and support action,"[{'rcn': '853', 'title': 'Specific Programme ""...",The objective of the EuroTraining proposal is ...,\nNext-Generation Nanoelectronics Components a...,2007-11-01,CLOSED,Provision of a European training infrastructure,1398009,http://www.eurotraining.net/


In [77]:
# add number of partners per project as a column
count_partners_df = cordis_project_orgs_df[['project_rcn', 'organization_id']].groupby( by='project_rcn' ).count()
count_partners_df.rename( columns={'organization_id': 'number_of_partners'}, inplace=True )

cordis_project_orgs_df = cordis_project_orgs_df.merge(
  count_partners_df, on='project_rcn', how='left'
)

# display header
cordis_project_orgs_df.head(2)


,project_rcn,organization_id,activity_type,address,ec_contribution_organization,type,website_x,id,name,country_code,country_name,rcn,acronym,end_date_code,ec_contribution_project,framework,funding_scheme,funded_under,objective,project_description,start_date_code,status,title,total_cost_project,website_y,number_of_partners
0,85231,999635926,Other,"{'city': 'TORINO', 'street': 'CORSO DUCA DEGLI...",306448,participant,www.corep.it,999635926,CONSORZIO PER LA RICERCA E L EDUCAZIONE PERMAN...,IT,Italy,85231,EuroTraining,2010-10-31,1398009,FP7,CSA - Coordination and support action,"[{'rcn': '853', 'title': 'Specific Programme ""...",The objective of the EuroTraining proposal is ...,\nNext-Generation Nanoelectronics Components a...,2007-11-01,CLOSED,Provision of a European training infrastructure,1398009,http://www.eurotraining.net/,5
1,85231,999665802,Private for-profit entities (excluding Higher ...,"{'city': 'HOLTE', 'street': 'FREDERIKSLUNDSVEJ...",396542,coordinator,,999665802,TECHNOCONSULT APS,DK,Denmark,85231,EuroTraining,2010-10-31,1398009,FP7,CSA - Coordination and support action,"[{'rcn': '853', 'title': 'Specific Programme ""...",The objective of the EuroTraining proposal is ...,\nNext-Generation Nanoelectronics Components a...,2007-11-01,CLOSED,Provision of a European training infrastructure,1398009,http://www.eurotraining.net/,5


# Playing with the data

In [82]:
cordis_project_orgs_df.columns

Index(['project_rcn', 'organization_id', 'activity_type', 'address',
       'ec_contribution_organization', 'type', 'website_x', 'id', 'name',
       'country_code', 'country_name', 'rcn', 'acronym', 'end_date_code',
       'ec_contribution_project', 'framework', 'funding_scheme',
       'funded_under', 'objective', 'project_description', 'start_date_code',
       'status', 'title', 'total_cost_project', 'website_y',
       'number_of_partners'],
      dtype='object')

In [70]:
cordis_project_orgs_df['activity_type'].value_counts()

Higher or Secondary Education Establishments                                                         88335
Private for-profit entities (excluding Higher or Secondary Education Establishments)                 78810
Research Organisations                                                                               55099
Other                                                                                                15406
Public bodies (excluding Research Organisations and Secondary or Higher Education Establishments)    12454
                                                                                                       936
Name: activity_type, dtype: int64

In [79]:
subset = cordis_project_orgs_df[
    (cordis_project_orgs_df.activity_type == 'Private for-profit entities (excluding Higher or Secondary Education Establishments)')
  & (cordis_project_orgs_df.framework == 'H2020')
]

# within subset, how often do organisations appear
subset['organization_id'].value_counts().head()



999993856    148
999960488     84
999909854     79
999951467     76
999908787     73
Name: organization_id, dtype: int64

In [83]:
subset[['organization_id', 'start_date_code', 'funding_scheme', 'number_of_partners', 'ec_contribution_organization', 'ec_contribution_project', 'total_cost_project']].head()

,organization_id,start_date_code,funding_scheme,number_of_partners,ec_contribution_organization,ec_contribution_project,total_cost_project
135899,999905974,2014-01-01,CSA - Coordination and support action,1,230000,230000,230000
135943,999574428,2014-08-01,CSA - Coordination and support action,38,20000,5000000,5000000
135953,999940118,2014-08-01,CSA - Coordination and support action,38,35000,5000000,5000000
135972,984286452,2014-10-01,CSA - Coordination and support action,58,31608,6000000,6000000
135977,998142126,2014-10-01,CSA - Coordination and support action,58,551450,6000000,6000000
